In [ ]:
import random

NUM = 100 # 每次生成100道题
TYPE_DICT = ['mul', 'div']

MUL1_MIN = 10
MUL1_MAX = 300
MUL2_MIN = 2
MUL2_MAX = 9

DIV1_MIN = 20
DIV1_MAX = 500
DIV2_MIN = 2
DIV2_MAX = 9

In [ ]:
type_rnd = random.randint(0, len(TYPE_DICT))
mul1 = random.randint(MUL1_MIN, MUL1_MAX)
mul2 = random.randint(MUL2_MIN, MUL2_MAX)

mul_res = mul1 * mul2

div1 = random.randint(DIV1_MIN, DIV1_MAX)
div2 = random.randint(DIV2_MIN, DIV2_MAX)

div_res = div1//div2

div_rem = div1 % div2

print(TYPE_DICT[type_rnd])
print(f"mul1 = {mul1}, mul2 = {mul2} mul_res = {mul_res}")
print(f"div1 = {div1}, div2 = {div2} div_res = {div_res} div_rem ={div_rem}")


In [ ]:
def mul_problem_gen (index:int, mul1:int, mul2:int, debug: bool = False):
    mul_res = mul1 * mul2
    format_str = f"{index:<4} | {mul1:<4} x {mul2:>4} = {'\0' * 4} |{'\0' *4} 答案：{mul_res}"
    if debug: print(format_str)
    return format_str
    

In [ ]:
mul_problem_gen(1,18,3, True)
mul_problem_gen(122,55,5, True)

In [ ]:
def div_problem_gen (index:int, div1:int, div2:int,  debug: bool = False):

    div_res = div1//div2

    div_rem = div1 % div2
    
    ans_str = f"{div_res}" if div_rem == 0 else f"{div_res}...{div_rem}" 
    
    format_str = f"{index:<4} | {div1:<4} \u00F7 {div2:>4} = {'\0' * 4} |{'\0' * 4} 答案：{ans_str}"
    if debug: print(format_str)
    return format_str

In [ ]:
div_problem_gen(1,18,3, True)
div_problem_gen(12,55,15, True)

In [ ]:
def problem_gen (type_: str, index: int):
    problem_str = None
    if type_ == 'mul':
        mul1 = random.randint(MUL1_MIN, MUL1_MAX)
        mul2 = random.randint(MUL2_MIN, MUL2_MAX)
        problem_str = mul_problem_gen(index, mul1, mul2)
    elif type_ == 'div':
        div1 = random.randint(DIV1_MIN, DIV1_MAX)
        div2 = random.randint(DIV2_MIN, DIV2_MAX)
        problem_str = div_problem_gen(index, div1, div2)
    return problem_str
        

In [ ]:
def print_problem_single(round_:int):
    all_out = ''
    for i in range (round_):
#         row = 0
        type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        
        print_str = problem_gen(type_, i+1)

        out = f"{print_str:<50}\n"
        
        print(out)

        all_out += out

    return all_out   


In [ ]:
def print_problem_double(round_:int):
    for i in range (round_):
        row = 0
        
        type_ = TYPE_DICT[random.randint(0, len(TYPE_DICT)-1)]
        
        print_str = problem_gen(type_, i+1)
        
        print(f"{print_str:<50}", end='|  ')
        if i % 2 == 1: #每行两个题目
            row += 1

            print("\n")  # 每两个题目换行
            print("\n")  # 每两个题目加一空行
            
        i += 1

In [ ]:
#print_problem_double(NUM)

In [ ]:
def print_problem_daily(days:int, nums:int):
    for d in range(days):
        print("日期_______， 正确数目_______, 错误数目_______, 得分_______")
        print_problem_single(nums)
        print("\n"*2)

In [ ]:
#print_problem_daily(10, 35)

In [ ]:
def gen_problem_per_day(nums:int):
    out=''
    out+="日期_______， 正确数目_______, 错误数目_______, 得分_______\n"
    out+=print_problem_single(nums)
    out+="\n"*2
    return out

In [ ]:
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

# pdfmetrics.registerFont(TTFont('SimSun', './SimSun.ttf'))
# pdfmetrics.registerFont(TTFont('AR PL UMing CN', 'uming.ttc'))
# styles = getSampleStyleSheet()

pdf_name = "multi_page_problem.pdf"

def print_problem_pdf(days, nums):
    pdfmetrics.registerFont(TTFont('SimSun', './SimSun.ttf'))  #注册字体
    styles = getSampleStyleSheet()
    styles.add(ParagraphStyle(fontName='SimSun', name='Song', leading=15, fontSize=13))  #leading=spacing
      
    doc = SimpleDocTemplate(pdf_name, pagesize=letter)
    
    # styles.add(ParagraphStyle(name='Chinese', fontName='AR PL UMing CN',    fontSize=20))
    styles.add(ParagraphStyle(name='Address', fontName='Arial'))
    flowables = []

    for d in range(days):
        # 将换行符替换为LineBreak对象
        problem_text = gen_problem_per_day(nums)
        lines = problem_text.split('\n')
        for line in lines:
            flowables.append(Paragraph(line, styles['Song']))

        # 添加额外的间距
        flowables.append(Spacer(1, 12))

        if d < days - 1:  # 除了最后一页外，其他每页末尾添加分页符
            flowables.append(PageBreak())

    doc.build(flowables)

# 使用函数生成多页PDF
days = 10  # 
nums = 40  # 
print_problem_pdf(days, nums)